## 参考
1. [Generic cryptographic module](https://www.pyopenssl.org/en/stable/api/crypto.html)
2. [Certificate Attributes](https://docs.oracle.com/cd/E24191_01/common/tutorials/authz_cert_attributes.html)
3. [源码](https://github.com/moses-palmer/truepy)

In [257]:
from OpenSSL import crypto, SSL
from cryptography.hazmat import backends
from cryptography.hazmat.primitives import serialization

from truepy import LicenseData, License
import datetime

from os import path
from lxml import etree
import time

import json
import ast

In [213]:
def generate_certificate(
    emailAddress="motein@qq.com",
    commonName="EnvisionNano Cert",
    countryName="CN",
    localityName="Lower Xiehe Street 888, Shuangliu District",
    stateOrProvinceName="Sichuan",
    organizationName="EnvisionNano, Inc.",
    organizationUnitName="R&D",
    serialNumber=0,
    validityStartInSeconds=0,
    validityEndInSeconds=9*365*24*60*60,
    KEY_FILE = "private.pem",
    CERT_FILE="certificate.pem"):
    #can look at generated file using openssl:
    #openssl x509 -inform pem -in certificate.pem -noout -text
    # create a key pair
    k = crypto.PKey()
    k.generate_key(crypto.TYPE_RSA, 4096)
    # create a self-signed cert
    cert = crypto.X509()
    cert.get_subject().C = countryName
    cert.get_subject().ST = stateOrProvinceName
    cert.get_subject().L = localityName
    cert.get_subject().O = organizationName
    cert.get_subject().OU = organizationUnitName
    cert.get_subject().CN = commonName
    cert.get_subject().emailAddress = emailAddress
    cert.set_serial_number(serialNumber)
    cert.gmtime_adj_notBefore(0)
    cert.gmtime_adj_notAfter(validityEndInSeconds)
    cert.set_issuer(cert.get_subject())
    cert.set_pubkey(k)
    cert.sign(k, 'sha512')
    tt = crypto.dump_certificate(crypto.FILETYPE_PEM, cert)
    print(type(tt),len(tt))
    with open(CERT_FILE, "wt") as f:
        f.write(crypto.dump_certificate(crypto.FILETYPE_PEM, cert).decode("utf-8"))
    with open(KEY_FILE, "wt") as f:
        f.write(crypto.dump_privatekey(crypto.FILETYPE_PEM, k).decode("utf-8"))

In [214]:
generate_certificate()

<class 'bytes'> 2110


In [215]:
with open('certificate.pem', 'rb') as f:
    certificate = f.read()

In [216]:
with open('private.pem', 'rb') as f:
    key = serialization.load_pem_private_key(
        f.read(),
        password=None,
        backend=backends.default_backend())

In [217]:
DAYS= 186
start_time = datetime.datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%S')
start_time_obj = datetime.datetime.strptime(start_time, '%Y-%m-%dT%H:%M:%S')
end_time_obj = start_time_obj+datetime.timedelta(days=DAYS)
end_time = end_time_obj.strftime('%Y-%m-%dT%H:%M:%S')

In [256]:
with open('certificate.pem', 'rb') as f:
    ff = f.read().splitlines()
    L = len(ff)
    dict_v = {}
    dict_v["size"] = L
    for i in range(L):
        dict_v[i] = ff[i].decode('utf-8')
    print(str(dict_v))
#     print(type(ff), L, ff)

{'size': 34, 0: '-----BEGIN CERTIFICATE-----', 1: 'MIIF6DCCA9ACAQAwDQYJKoZIhvcNAQENBQAwgbkxCzAJBgNVBAYTAkNOMRAwDgYD', 2: 'VQQIDAdTaWNodWFuMTMwMQYDVQQHDCpMb3dlciBYaWVoZSBTdHJlZXQgODg4LCBT', 3: 'aHVhbmdsaXUgRGlzdHJpY3QxGzAZBgNVBAoMEkVudmlzaW9uTmFubywgSW5jLjEM', 4: 'MAoGA1UECwwDUiZEMRowGAYDVQQDDBFFbnZpc2lvbk5hbm8gQ2VydDEcMBoGCSqG', 5: 'SIb3DQEJARYNbW90ZWluQHFxLmNvbTAeFw0yMDEyMDcxNzA2MjJaFw0yOTEyMDUx', 6: 'NzA2MjJaMIG5MQswCQYDVQQGEwJDTjEQMA4GA1UECAwHU2ljaHVhbjEzMDEGA1UE', 7: 'BwwqTG93ZXIgWGllaGUgU3RyZWV0IDg4OCwgU2h1YW5nbGl1IERpc3RyaWN0MRsw', 8: 'GQYDVQQKDBJFbnZpc2lvbk5hbm8sIEluYy4xDDAKBgNVBAsMA1ImRDEaMBgGA1UE', 9: 'AwwRRW52aXNpb25OYW5vIENlcnQxHDAaBgkqhkiG9w0BCQEWDW1vdGVpbkBxcS5j', 10: 'b20wggIiMA0GCSqGSIb3DQEBAQUAA4ICDwAwggIKAoICAQDITLurLb/HIpOWcojK', 11: '8fMf2b83ndgXoUuvoVMD1FcNy1qZBrqEZHbTOaNzg+CdclkG9loCoOAzYnilS9rl', 12: 'ohlgbqK5rnHlV2kD0z1WrGWa8e+h/PCiXa4N4FTIIWlfMlODy+6ABPOfhBbgZmFd', 13: 'SVqGE2lC0UxSkv7YtwENH7cuA/KtVDeJBFavdazogzh1C5gRKMU3Ra+pAHJ6/CHc', 14: 'OSulL0dHf039aKT8A13i3

In [219]:
extra_info = str(dict_v)
extra_info

"{'size': 34, 0: '-----BEGIN CERTIFICATE-----', 1: 'MIIF6DCCA9ACAQAwDQYJKoZIhvcNAQENBQAwgbkxCzAJBgNVBAYTAkNOMRAwDgYD', 2: 'VQQIDAdTaWNodWFuMTMwMQYDVQQHDCpMb3dlciBYaWVoZSBTdHJlZXQgODg4LCBT', 3: 'aHVhbmdsaXUgRGlzdHJpY3QxGzAZBgNVBAoMEkVudmlzaW9uTmFubywgSW5jLjEM', 4: 'MAoGA1UECwwDUiZEMRowGAYDVQQDDBFFbnZpc2lvbk5hbm8gQ2VydDEcMBoGCSqG', 5: 'SIb3DQEJARYNbW90ZWluQHFxLmNvbTAeFw0yMDEyMDcxNzA2MjJaFw0yOTEyMDUx', 6: 'NzA2MjJaMIG5MQswCQYDVQQGEwJDTjEQMA4GA1UECAwHU2ljaHVhbjEzMDEGA1UE', 7: 'BwwqTG93ZXIgWGllaGUgU3RyZWV0IDg4OCwgU2h1YW5nbGl1IERpc3RyaWN0MRsw', 8: 'GQYDVQQKDBJFbnZpc2lvbk5hbm8sIEluYy4xDDAKBgNVBAsMA1ImRDEaMBgGA1UE', 9: 'AwwRRW52aXNpb25OYW5vIENlcnQxHDAaBgkqhkiG9w0BCQEWDW1vdGVpbkBxcS5j', 10: 'b20wggIiMA0GCSqGSIb3DQEBAQUAA4ICDwAwggIKAoICAQDITLurLb/HIpOWcojK', 11: '8fMf2b83ndgXoUuvoVMD1FcNy1qZBrqEZHbTOaNzg+CdclkG9loCoOAzYnilS9rl', 12: 'ohlgbqK5rnHlV2kD0z1WrGWa8e+h/PCiXa4N4FTIIWlfMlODy+6ABPOfhBbgZmFd', 13: 'SVqGE2lC0UxSkv7YtwENH7cuA/KtVDeJBFavdazogzh1C5gRKMU3Ra+pAHJ6/CHc', 14: 'OSulL0dHf039aKT8A13i

In [220]:

# Issue the license
local_license = License.issue(
    certificate,
    key,
    license_data=LicenseData(start_time, end_time, info=extra_info))

In [221]:
with open('license.key', 'wb') as f:
    local_license.store(f, b'motein@XA')

In [269]:
def get_certificate(cert='certificate.pem'):
    with open(cert, 'rb') as f:
        certificate = f.read()
        return certificate

def is_expired(local_license):
    root = etree.fromstring(local_license.encoded.encode('ascii'))
#     print(local_license.encoded)
    time_value = []
    for time_element in root.findall('object/void/object/long'):
        time_value.append(int(time_element.text))
    if (time.time() * 1000) > max(time_value):
        return True
    else:
        return False

def get_certificate_from_license(local_license):
    root = etree.fromstring(local_license.encoded.encode('ascii'))
    cert_json = None
    for time_element in root.findall('object/void'):
        if time_element.attrib['property'] == 'info':
            child = time_element.find('string')
            cert_json = child.text
            break
#     print(type(cert_json), cert_json)
    res = ast.literal_eval(cert_json)
#     print(type(res), res)
    
    store_cert = ""
    for i in range(res['size']):
        store_cert = store_cert + res[i] + '\t\n'
        
#     print(store_cert)
    return store_cert.encode('utf-8')

def is_valid_license(license_path='license.key'):
    if path.exists(license_path) is False:
        return False, 'License文件不存在'
        
#     certificate = get_certificate()
    with open(license_path, 'rb') as f:
        try:
            local_license = License.load(f, b'motein@XA')
            certificate = get_certificate_from_license(local_license)
            local_license.verify(certificate)
        except (License.InvalidSignatureException):
            return False, '非法的License'
        except Exception:
            return False, '验证License失败'
        else:
            if is_expired(local_license):
                return False, 'License已过期'
            else:
                return True, '合法的License'

In [270]:
is_valid_license()

(True, '合法的License')